---
# Seq2Seq

- https://medium.com/@dev.elect.iitd/neural-machine-translation-using-word-level-seq2seq-model-47538cba8cd7
- https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py
- https://machinelearningmastery.com/define-encoder-decoder-sequence-sequence-model-neural-machine-translation-keras/
- https://machinelearningmastery.com/develop-encoder-decoder-model-sequence-sequence-prediction-keras/
- https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
- https://www.tensorflow.org/tutorials/text/nmt_with_attention

---



- **`encoder`: it processes the input sequence and returns its own internal state.** 

    - encoder RNN의 아웃풋은 버린다! only recovering the state.
    - This state will serve as the "context", or "conditioning"      
    
    
- **`decoder`: it is trained to predict the next characters of the target sequence, given previous characters of the target sequence.** 
    - it is trained to **turn the target sequences into the same sequences** 
    - but offset by one timestep in the future, 
    - a training process called "teacher forcing" in this context. 
    
    - `Importantly, the encoder uses as initial state the state vectors from the encoder, which is how the decoder obtains information about what it is supposed to generate.` 
    - Effectively, the decoder learns to generate targets[t+1...] given targets[...t], conditioned on the input sequence.

In [53]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import re
import numpy as np
import pandas as pd
import os
import io
import time

# Load Data

In [36]:
# load
with open('dataset/df_story_desc_final.pickle', 'rb') as f:
    desc_story = pickle.load(f)

print ("before: ", desc_story.shape)
desc_story = desc_story

desc_story_id = desc_story['Story_Photo_id']
desc_story_text = desc_story[['Desc', 'Story']]

print ("after: ", desc_story_text.shape)
desc_story_text.head(15)

before:  (145116, 4)
after:  (145116, 2)


,Desc,Story
0,big old tree being photographed on a sunny day,"and its magnificent trunk , larger than life i..."
1,a old curvy tree in the sun light .,"and its magnificent trunk , larger than life i..."
2,a person is taking a picture of a large tree a...,"and its magnificent trunk , larger than life i..."
3,large tree with many outstretching branches an...,we found this tree when we were walking in a n...
4,a green sign is describing a historic tree and...,it turns out it is a popular attraction here .
5,a large tree with roots that look like crocodi...,"the tree is very unusual , with its roots expo..."
6,big old tree being photographed on a sunny day,"the trunk was really wide , as much as 12 feet !"
7,huge brown tree roots rose above the ground .,you can see how big these roots are - pretty a...
8,a large tree with many branches coming out,we found this tree when we were walking in a n...
9,a plaque describes an historical tree and advi...,it turns out it is a popular attraction here .


==> 15개 단위로 같은 사진 set에 대한 내용

In [37]:
desc_story_text[desc_story_text.Desc=='big old tree being photographed on a sunny day']

,Desc,Story
0,big old tree being photographed on a sunny day,"and its magnificent trunk , larger than life i..."
6,big old tree being photographed on a sunny day,"the trunk was really wide , as much as 12 feet !"
18,big old tree being photographed on a sunny day,some more different parts of the tree .
24,big old tree being photographed on a sunny day,the trunk was incredibly thick and rigid .
39,big old tree being photographed on a sunny day,i was dwarfed by the tree 's size .


In [38]:
desc_story_text[desc_story_text.Desc=='a old curvy tree in the sun light .']

,Desc,Story
1,a old curvy tree in the sun light .,"and its magnificent trunk , larger than life i..."
11,a old curvy tree in the sun light .,"the trunk was really wide , as much as 12 feet !"
19,a old curvy tree in the sun light .,some more different parts of the tree .
29,a old curvy tree in the sun light .,the trunk was incredibly thick and rigid .
44,a old curvy tree in the sun light .,i was dwarfed by the tree 's size .


In [39]:
for i,row in desc_story_text[desc_story_text.Desc=='big old tree being photographed on a sunny day'].iterrows():
    print("pair",i,":",row['Desc']+" ==> "+row['Story'])

pair 0 : big old tree being photographed on a sunny day ==> and its magnificent trunk , larger than life itself .
pair 6 : big old tree being photographed on a sunny day ==> the trunk was really wide , as much as 12 feet !
pair 18 : big old tree being photographed on a sunny day ==> some more different parts of the tree .
pair 24 : big old tree being photographed on a sunny day ==> the trunk was incredibly thick and rigid .
pair 39 : big old tree being photographed on a sunny day ==> i was dwarfed by the tree 's size .


==> 하나의 Description Text에 5개의 Story가 대응됨

---
번역 task 라고 생각하면, 하나의 문장에 다양한 해석이 나오면 성능이 안 좋을 수 있다   
===> 겹치는 문장 제거

In [46]:
def drop_duplicate(df):
    drop_df = df.drop_duplicates("Desc")
    drop_df = drop_df.drop_duplicates("Story")
    drop_df = drop_df.reset_index(drop=True)
    return drop_df

In [47]:
desc_story_text = drop_duplicate(desc_story_text)
desc_story_text.shape

(13307, 2)

# Prepare Data

## preprocess text

- Add a start and end token to each sentence.
- Clean the sentences by removing special characters.

In [48]:
def preprocess_sentence(w):
    w = re.sub('[^a-zA-Z]+', ' ', w)
    w = re.sub('[^a-zA-Z.,!?]+', ' ', w)
    w = w.strip()
    
    # adding a start and an end token to the sentence
    w = '<start> ' + w + ' <end>'
    return w

In [51]:
clean_data = desc_story_text.copy()
clean_data['Desc'] = desc_story_text['Desc'].apply(lambda x: preprocess_sentence(x))
clean_data['Story'] = desc_story_text['Story'].apply(lambda x: preprocess_sentence(x))
clean_data.columns = ['in_desc','out_story']

## Indexing and Padding

- Create a word index and reverse word index (dictionaries mapping from word → id and id → word).   
: 단어 색인과 역방향 단어 색인을 만듭니다 (단어 → ID 및 ID → 단어에서 매핑 된 사전).

- Pad each sentence to a maximum length.

In [122]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
      
    tensor = lang_tokenizer.texts_to_sequences(lang)
    # sentence 최대 길이로 패딩 자동으로 해줌
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    return tensor, lang_tokenizer

In [123]:
tensor, lang_tokenizer = tokenize(clean_data['in_desc'].tolist()[:5])

In [134]:
def load_tokenize_data(data):
    # creating cleaned input, output pairs
    inp_lang, targ_lang = data['in_desc'], data['out_story']

    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [135]:
input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer = load_tokenize_data(clean_data)

In [136]:
input_tensor

array([[  2, 201,  94, ...,   0,   0,   0],
       [  2,  21, 120, ...,   0,   0,   0],
       [  2,   1,  55, ...,   0,   0,   0],
       ...,
       [  2,   1,  11, ...,   0,   0,   0],
       [  2,   1,  16, ...,   0,   0,   0],
       [  2,  13, 495, ...,   0,   0,   0]], dtype=int32)

In [137]:
target_tensor

array([[  1,   7, 372, ...,   0,   0,   0],
       [  1,   9, 136, ...,   0,   0,   0],
       [  1,  11, 798, ...,   0,   0,   0],
       ...,
       [  1,  60, 485, ...,   0,   0,   0],
       [  1,   4, 157, ...,   0,   0,   0],
       [  1,  60, 539, ...,   0,   0,   0]], dtype=int32)

In [138]:
# Calculate max_length of the target tensors
max_length_targ, max_length_inp= target_tensor.shape[1], input_tensor.shape[1]

In [139]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

10645 10645 2662 2662


In [140]:
def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print ("%d ----> %s" % (t, lang.index_word[t]))

In [146]:
print ("Input Language; index to word mapping")
convert(inp_lang_tokenizer, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang_tokenizer, target_tensor_train[0])

Input Language; index to word mapping
2 ----> <start>
1 ----> a
5255 ----> solder
338 ----> talking
6 ----> in
28 ----> front
5 ----> of
1 ----> a
124 ----> crowd
60 ----> near
1 ----> a
1425 ----> national
1819 ----> guard
890 ----> poster
3 ----> <end>

Target Language; index to word mapping
1 ----> <start>
3638 ----> sergeant
29 ----> male
492 ----> takes
3 ----> the
500 ----> stage
7 ----> and
5305 ----> outlines
112 ----> what
11 ----> it
16 ----> is
5 ----> to
51 ----> be
4 ----> a
1457 ----> leader
2 ----> <end>


## Create a tf.data dataset

In [169]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang_tokenizer.word_index)+1
vocab_tar_size = len(targ_lang_tokenizer.word_index)+1

# 텐서플로우 데이터 형태로 만드는 코드..
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [170]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 51]), TensorShape([64, 80]))

# Write the encoder and decoder model

**why decoder_target_data.shape is 3d**

- 모든 단어에 대하여 이전 단어로부터 다음 단어를 예측하는 소프트맥스 층을 가지기 때문에 

--- 

0908 DONE

- 왜 1로 초기화 하는가

    - ==> 초기화를 1로 하는 것이 아니라 t=0을 건너뛰는 것임

---

In [109]:
input_token_index = dict([(word, i) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i) for i, word in enumerate(target_words)])

In [110]:
for i, (input_text, target_text) in enumerate(zip(clean_data['in_desc'], clean_data['out_story'])):

    # encoder
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]

    # decoder
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]  
        if t > 0: 
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [111]:
encoder_input_data.shape

(13307, 51)

In [112]:
print (decoder_input_data.shape)
decoder_input_data

(13307, 80)


array([[1.000e+00, 2.160e+02, 3.680e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.000e+00, 7.587e+03, 2.795e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.000e+00, 3.676e+03, 7.284e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [1.000e+00, 1.180e+02, 2.203e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.000e+00, 2.000e+00, 2.592e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.000e+00, 1.180e+02, 6.245e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00]], dtype=float32)

In [17]:
print(decoder_target_data.shape)
decoder_target_data

(8255, 81, 3463)


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

--- 

0909 DONE

---

# Build keras encoder-decoder model

http://incredible.ai/nlp/2020/02/20/Sequence-To-Sequence-with-Attention/
https://docs.chainer.org/en/stable/examples/seq2seq.html

https://tykimos.github.io/2018/09/14/ten-minute_introduction_to_sequence-to-sequence_learning_in_Keras/

In [18]:
import tensorflow as tf
from tensorflow import keras

from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Bidirectional

In [19]:
batch_size = 64  # Batch size for training.
epochs = 50  # Number of epochs to train for.
latent_dim = 64  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
embedding_size = 50 # Embedding Dimension

In [20]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
print('encoder_inputs: ', encoder_inputs.shape)

# English words embedding
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
print('en_x: ', en_x.shape)

encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
print('encoder_outputs: ', encoder_outputs.shape)
print('state_h: ', state_h.shape)
print('state_c: ', state_c.shape)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

encoder_inputs:  (None, None)
en_x:  (None, None, 50)
encoder_outputs:  (None, 64)
state_h:  (None, 64)
state_c:  (None, 64)


In [21]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, ))
print('decoder_inputs: ', decoder_inputs.shape)

# french word embeddings
dex=  Embedding(num_decoder_tokens, embedding_size)
final_dex = dex(decoder_inputs)

# We set up our decoder to return full output sequences, and to return internal states as well. 
# We don't use the return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
print('decoder_outputs: ', decoder_outputs.shape)

decoder_inputs:  (None, None)
decoder_outputs:  (None, None, 3463)


--- 

0911 DONE

---

In [22]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     282700      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     173150      input_2[0][0]                    
______________________________________________________________________________________________

In [157]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(8255, 53)
(8255, 81)
(8255, 81, 3463)


In [23]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [159]:
with tf.device('/gpu:0'):
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.2)

Epoch 1/50
104/104 [==============================] - 59s 568ms/step - loss: 0.9069 - accuracy: 0.5221 - val_loss: 0.8622 - val_accuracy: 0.8703
Epoch 2/50
104/104 [==============================] - 57s 548ms/step - loss: 0.8376 - accuracy: 0.8710 - val_loss: 0.8519 - val_accuracy: 0.8723
Epoch 3/50
104/104 [==============================] - 57s 550ms/step - loss: 0.8169 - accuracy: 0.8714 - val_loss: 0.8479 - val_accuracy: 0.8725
Epoch 4/50
104/104 [==============================] - 57s 553ms/step - loss: 0.8012 - accuracy: 0.8711 - val_loss: 0.8447 - val_accuracy: 0.8694
Epoch 5/50
104/104 [==============================] - 58s 553ms/step - loss: 0.7889 - accuracy: 0.8683 - val_loss: 0.8350 - val_accuracy: 0.8701
Epoch 6/50
104/104 [==============================] - 57s 547ms/step - loss: 0.7701 - accuracy: 0.8664 - val_loss: 0.8286 - val_accuracy: 0.8598
Epoch 7/50
104/104 [==============================] - 58s 558ms/step - loss: 0.7561 - accuracy: 0.8658 - val_loss: 0.8207 - val_ac

결과가.. 이상..

In [43]:
# Save model
model.save('s2s.h5')

In [24]:
model = keras.models.load_model('s2s.h5')

ValueError: Unknown layer: Functional

# Sentence Generation

In [131]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

Model: "functional_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_31 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_10 (Embedding)     (None, None, 50)          284500    
_________________________________________________________________
lstm_14 (LSTM)               [(None, 64), (None, 64),  29440     
Total params: 313,940
Trainable params: 313,940
Non-trainable params: 0
_________________________________________________________________


In [132]:
# Redefine the decoder model with decoder will be getting below inputs from encoder while in prediction
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)
print(final_dex2)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

# sampling model will take encoder states and decoder_input(seed initially) and output the predictions(french word index) We dont care about decoder_states2
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

Tensor("embedding_11/embedding_lookup_1/Identity_1:0", shape=(None, None, 50), dtype=float32)


In [133]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [134]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['<sos>']
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
       
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char
        # Exit condition: either hit max length or find stop character.
        if (sampled_char == '<eos>' or
           len(decoded_sentence) > 52):
            stop_condition = True
        
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        
        # Update states
        states_value = [h, c]
    return decoded_sentence

In [135]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', clean_data['in_desc'][seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: big old tree being photographed on a sunny day
Decoded sentence:  and i m going to be graduating he could n t wait for
-
Input sentence: a old curvy tree in the sun light 
Decoded sentence:  and i m going to be graduating he could n t wait for
-
Input sentence: a person is taking a picture of a large tree and you can see their shadow
Decoded sentence:  and the dog was the dog and have a panda it <eos>
-
Input sentence: large tree with many outstretching branches and leaves 
Decoded sentence:  we had a great time for the city <eos>
-
Input sentence: a green sign is describing a historic tree and the rules for visiting it 
Decoded sentence:  we had a great time <eos>
-
Input sentence: a large tree with roots that look like crocodile tails 
Decoded sentence:  the family was on a great time <eos>
-


KeyError: 6

---
- epoch 를 줄여야 어느정도 학습 가능
- input에 대응하는 output pair 가 너무 많아서 성능이 안좋은게아닐까? => input : output 1대1 대응만 남기기